In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from uuid import UUID
from sqlalchemy.ext.asyncio import AsyncSession
from chat import MeetingChatManager
from psql_helpers import get_session

In [3]:
from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()
user_id = vexa.user_id


Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [4]:
from uuid import UUID

In [5]:


# Single meeting chat
meeting_id = UUID('aa87b543-f5c9-426c-a6ed-77b26f93e0da')
async with get_session() as session:
    chat_manager = MeetingChatManager(session)
    await chat_manager.initialize()
    
    # Chat about specific meeting - will create meeting-specific thread
    async for result in chat_manager.chat(
        user_id=user_id,
        meeting_ids=[meeting_id],
        query="50 char on the meeting"
    ):
        if "chunk" in result:
            print(result["chunk"], end="")
        elif "thread_id" in result:
            thread_id = result["thread_id"]  # Store for continued conversation

16:05:46 - vexa_api - INFO - Initializing ChatManager


Dmitriy completed API; frontend integration ongoing.

In [6]:
thread_id

'cf75f12e-5205-434a-abad-6e9540ededf1'

In [7]:
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=[meeting_id],
    thread_id=thread_id,
    query="What were the key decisions?"
):
    print(result.get("chunk", ""), end="")

- **API Development Completion**: Dmitriy Grankin confirmed the completion of the API, which is now ready for frontend integration.

- **Frontend Integration**: Ongoing work on integrating the frontend with the API, with a focus on ensuring elements are clickable and functional.

- **Model Testing**: Decision to test the LAMA 3.2 model with 90 billion parameters, which has shown promising results but requires further refinement.

- **Authorization Method**: Agreement to implement Google authorization for user access, utilizing LocalStorage for storing user tokens.

- **Error Handling**: Acknowledgment of issues related to 'incomplete chunk encoding' errors, necessitating further investigation and resolution.

- **Data Indexing**: Plan to index data on the dashboard to facilitate the display of information related to calls and chats, enhancing user experience.

- **Design Considerations**: Discussion on the importance of adhering to a design system for the frontend, ensuring consistency

In [8]:
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=[meeting_id],
    thread_id=thread_id,
    query="in a few words"
):
    print(result.get("chunk", ""), end="")

Dmitriy completed API; frontend integration ongoing; testing LAMA 3.2 model; addressing authorization and error handling issues.

In [ ]:
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=[meeting_id],
    thread_id=thread_id,
    query=""
):
    print(result.get("chunk", ""), end="")

In [13]:
from chat import SearchChatManager

In [14]:

async with get_session() as session:
    chat_manager = SearchChatManager(session)
    await chat_manager.initialize()

TypeError: ChatManager.__init__() takes 1 positional argument but 2 were given

In [ ]:
# Follow up in global chat
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=None,
    thread_id=None,
    query="Can you help me analyze meetings?"
):
    print(result.get("chunk", ""), end="")

In [ ]:

# Chat about multiple meetings
async for result in chat_manager.chat(
    user_id=user_id,
    meeting_ids=[UUID('id1'), UUID('id2')],
    query="Compare these meetings"
):
    print(result.get("chunk", ""), end="")